In [ ]:
%pip install requests
%pip install pprint

In [ ]:
BASE_URL = "http://localhost:8000"
API_URL = f"{BASE_URL}/api"
ACCESS_TOKEN="<token>"

In [ ]:
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

In [ ]:
# create organization
import requests
from starlette import status

org_id = ""
cfg = {}

payload = {
    "display_name": "ACME, inc.",
    "config": cfg
}

resp = requests.post(f"{API_URL}/orgs", headers=headers, json=payload)

if resp.status_code != status.HTTP_201_CREATED:
    raise SystemExit(resp.text)

j = resp.json()
print("org created: ", j)
org_id = j['id']

In [ ]:
# get organization
import requests
from starlette import status
resp = requests.get(f"{API_URL}/orgs/1234", headers=headers)
if resp.status_code != status.HTTP_404_NOT_FOUND:
    raise SystemExit(f"expected code 404 but got {resp.status_code}")
resp = requests.get(f"{API_URL}/orgs/{org_id}", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
print("org retrieved: ", resp.json())

In [ ]:
# add content to org' vectorstore
import requests

try:
  content = """**user:erezsh**: Hi, I'm trying to run jobs in bigquery and I'm getting 403 error
          **IT**: What's the api you're trying to use?
          **user:erezsh**: `POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/my_project_id/jobs`
          **IT**: Looks like you need an iam role. I'll give you the role **gcp/roles/bigquery.admin**. You can try again now.
          **user:erezsh**: That worked, thanks."""
  
  data = {
    "docs": 
      [
        {
           "content": content,
           "metadata": {"foo": "bar"},
           "id": "1234",
        }
      ]
  }

  response = requests.post(f"{API_URL}/content", headers=headers, json=data)
  if response.status_code >= 200 and response.status_code <= 399:
      print("content successfully added: ", response.text)
  else:
      print(f"failed to add content with code: {response.status_code} || {response.text}")

except requests.RequestException as e:
  print(f"An error occurred: {e}")

In [ ]:
# remove data
import requests

try:
  data = {
     "ids": ["4"]
  }

  response = requests.post(f"{API_URL}/content/.delete", headers=headers, json=data)
  if response.status_code >= 200 and response.status_code <= 399:
      print("content successfully removed: ", response.text)
  else:
      print(f"failed to add content with code: {response.status_code} || {response.text}")

except requests.RequestException as e:
  print(f"An error occurred: {e}")

In [ ]:
# create conversation
import requests
from starlette import status


payload = {
    
}

resp = requests.post(f"{API_URL}/conversations", headers=headers, json=payload)

if resp.status_code != status.HTTP_201_CREATED:
    raise SystemExit(resp.text)

j = resp.json()
print("conversation created: ", j)
conversation_id = j['id']

In [ ]:
# send a message to conversation
import requests
from sseclient import SSEClient

def converse(input: str):
    data = {
        "input": input
    }
    _headers = headers.copy()
    _headers["Accept"] = "text/event-stream"
    with requests.post(
        url=f"{API_URL}/conversations/{conversation_id}/conversation/stream",
        json=data,
        headers=_headers,
        stream=True
    ) as r:
        if r.status_code != 200:
            raise SystemExit(f"expected status 200 but got {r.status_code}: {r.text}")

        client = SSEClient(r)
        buf = []
        for event in client.events():
            c = event.data
            buf.append(c)
            print(c, end='', flush=True)
        print("\n--")

try:
    input = "I need access for bigquery app"
    converse(input)

    # input = "I need to run jobs"
    # converse(input)
    
except requests.RequestException as e:
    print(f"error: {e}")

In [ ]:
# get conversation
import requests
from starlette import status
from pprint import pprint
    
resp = requests.get(f"{API_URL}/requests/404", headers=headers)
if resp.status_code != status.HTTP_404_NOT_FOUND:
    raise SystemExit(f"expected code 404 but got {resp.status_code}")
    
resp = requests.get(f"{API_URL}/conversations/{conversation_id}?links=messages", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
pprint(resp.json())
